<a href="https://colab.research.google.com/github/HuyKhoi-code/fruits/blob/master/test_color%2Bedge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score

In [ ]:
def clustering_image(img):
  #img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  vectorized = img.reshape((-1,3))
  vectorized = np.float32(vectorized)
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  K = 2
  attempts=10
  ret,label,center=cv2.kmeans(vectorized,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
  center = np.uint8(center)
  res = center[label.flatten()]
  result_image = res.reshape((img.shape))
  return result_image, center

In [ ]:
def remove_light_color(img, center_color):
  light_color = max(sum(center_color[0]), sum(center_color[1]))
  for x in range(img.shape[0]):
    for y in range(img.shape[1]):
      if sum(img[x][y]) == light_color:
        img[x][y][0]=0
        img[x][y][1]=0
        img[x][y][2]=0
  return img

In [ ]:
def Canny(img):
    img, center = clustering_image(img)
    img = remove_light_color(img, center)
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(img, (3,3), 0)
    edge = cv2.Canny(blur, 70 ,50)
    return edge

In [ ]:
def features_grid(img):
    features = np.array([], dtype='uint8')
    section = 1
    
    for y in range(0, img.shape[0], 40):
        for x in range(0, img.shape[1], 40):
            
            # Cropping the image into a section.
            section_img = img[y:y+40, x:x+40]
            
            # Claculating the mean and stdev of the sectioned image.
            section_mean = np.mean(section_img)
            section_std = np.std(section_img)
            
            # Appending the above calculated values into features array.
            features = np.append(features, [section_mean, section_std])
    
    # Returning the features array.
    return features

In [ ]:
def create_df_with_one_image1(img):
  columns_name = ['red_mean', 'red_std', 'green_mean', 'green_std', 'blue_mean', 'blue_std']
  red_means = []
  red_stds = []
  green_means = []
  green_stds = []
  blue_means = []
  blue_stds = []

  red_channel = img[:,:,0]
  green_channel = img[:,:,1]
  blue_channel = img[:,:,2]

  red_mean = np.mean(red_channel)
  green_mean = np.mean(green_channel)
  blue_mean = np.mean(blue_channel)
  
  red_std = np.std(red_channel)
  green_std = np.std(green_channel)
  blue_std = np.std(blue_channel)

  red_means.append(red_mean)
  red_stds.append(red_std)
  green_means.append(green_mean)
  green_stds.append(green_std)
  blue_means.append(blue_mean)
  blue_stds.append(blue_std)

      
  data = {'red_mean': red_means, 'red_std': red_stds, 'green_mean': green_means, 
          'green_std': green_stds, 'blue_mean': blue_means, 'blue_std': blue_stds}
  df = pd.DataFrame(data)
  return df

In [ ]:
def create_df_with_one_image2(img):
  feature_name = []
  section = 1
  for y in range(0, 200, 40):
      for x in range(0, 200, 40):
          feature_name.append(f"sec{section}_mean")
          feature_name.append(f"sec{section}_std")
          section += 1
  
  all_imgs = np.zeros((1, 50), dtype='uint8')

  name = []
  section_mean = []
  section_std = []


  img_features = features_grid(img)
  img_features = img_features.reshape(1, img_features.shape[0])

  all_imgs = np.append(all_imgs, img_features, axis=0)
  all_imgs = all_imgs[1:]

  df = pd.DataFrame(all_imgs, columns= feature_name)
  return df

In [ ]:
train_edge = pd.read_csv('/content/drive/My Drive/Fruit recognition/output_input/25_edge+cluster_train_features.csv')
valid_edge = pd.read_csv ('/content/drive/My Drive/Fruit recognition/output_input/25_edge+cluster_valid_features.csv')

In [ ]:
train_color = pd.read_csv('/content/drive/My Drive/Fruit recognition/output_input/25_cluster_features_2_train.csv')
valid_color = pd.read_csv('/content/drive/My Drive/Fruit recognition/output_input/25_cluster_features_2_test.csv')

In [ ]:
train_color

,name,red_mean,red_std,green_mean,green_std,blue_mean,blue_std
0,cachua,33.267025,58.744255,13.598200,24.012250,13.355375,23.583460
1,cachua,60.434000,83.571022,22.319375,30.864298,17.855500,24.691438
2,cachua,24.660350,54.021266,11.554150,25.310663,10.864350,23.799579
3,cachua,30.191525,62.843515,10.501400,21.858614,8.251100,17.174625
4,cachua,37.891425,63.108745,17.488350,29.127113,16.428450,27.361833
...,...,...,...,...,...,...,...
1865,xoai,32.681250,55.224753,36.831250,62.237421,26.975000,45.582336
1866,xoai,30.282600,52.987960,33.975600,59.449907,24.866200,43.510439
1867,xoai,27.335700,51.933204,30.589950,58.115729,23.213650,44.102007
1868,xoai,33.960500,55.297255,38.890250,63.324276,28.209125,45.932398


In [ ]:
train = pd.concat([train_color, train_edge.drop(['name'], axis=1)],axis=1)
valid = pd.concat([valid_color, valid_edge.drop(['name'], axis=1)],axis=1)

In [ ]:
train

,name,red_mean,red_std,green_mean,green_std,blue_mean,blue_std,sec1_mean,sec1_std,sec2_mean,sec2_std,sec3_mean,sec3_std,sec4_mean,sec4_std,sec5_mean,sec5_std,sec6_mean,sec6_std,sec7_mean,sec7_std,sec8_mean,sec8_std,sec9_mean,sec9_std,sec10_mean,sec10_std,sec11_mean,sec11_std,sec12_mean,sec12_std,sec13_mean,sec13_std,sec14_mean,sec14_std,sec15_mean,sec15_std,sec16_mean,sec16_std,sec17_mean,sec17_std,sec18_mean,sec18_std,sec19_mean,sec19_std,sec20_mean,sec20_std,sec21_mean,sec21_std,sec22_mean,sec22_std,sec23_mean,sec23_std,sec24_mean,sec24_std,sec25_mean,sec25_std
0,cachua,33.267025,58.744255,13.598200,24.012250,13.355375,23.583460,0.00000,0.000000,0.000000,0.000000,7.650000,43.499741,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.418750,36.775242,24.543750,75.208115,10.359375,50.342070,0.000000,0.000000,0.000000,0.000000,6.375000,39.811862,0.000000,0.000000,6.375000,39.811862,0.000000,0.000000,0.000000,0.000000,9.881250,49.214628,0.000000,0.000000,10.837500,51.440364,0.000000,0.000000,0.000000,0.000000,2.390625,24.574261,10.518750,50.711312,4.462500,33.436860,0.000000,0.000000
1,cachua,60.434000,83.571022,22.319375,30.864298,17.855500,24.691438,0.00000,0.000000,0.956250,15.586191,11.634375,53.210966,3.346875,29.021570,0.000000,0.000000,2.709375,26.144787,8.287500,45.217583,14.821875,59.664815,14.025000,58.134967,3.665625,30.352884,16.893750,63.423241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.375000,39.811862,0.000000,0.000000,9.881250,49.214628,3.346875,29.021570,9.243750,47.662452,2.550000,25.372180,5.578125,37.300220,32.034375,84.513694,47.812500,99.529656,4.303125,32.844786,0.000000,0.000000
2,cachua,24.660350,54.021266,11.554150,25.310663,10.864350,23.799579,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.534375,40.293518,11.475000,52.862552,4.940625,35.148963,0.000000,0.000000,0.000000,0.000000,6.375000,39.811862,0.000000,0.000000,6.375000,39.811862,0.000000,0.000000,0.000000,0.000000,6.056250,38.828670,7.650000,43.499741,10.996875,51.800308,6.375000,39.811862,0.000000,0.000000,1.275000,17.986088,7.331250,42.611284,0.478125,11.031467,0.000000,0.000000
3,cachua,30.191525,62.843515,10.501400,21.858614,8.251100,17.174625,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.968750,44.368122,0.000000,0.000000,8.287500,45.217583,6.375000,39.811862,6.375000,39.811862,2.231250,23.748479,3.984375,31.624996,7.809375,43.936366,0.000000,0.000000,8.128125,44.795150,0.000000,0.000000,9.403125,48.055989,9.243750,47.662452,10.996875,51.800308,5.737500,37.817239,0.000000,0.000000
4,cachua,37.891425,63.108745,17.488350,29.127113,16.428450,27.361833,0.00000,0.000000,0.000000,0.000000,7.650000,43.499741,4.303125,32.844786,0.000000,0.000000,0.000000,0.000000,4.303125,32.844786,14.025000,58.134967,15.618750,61.146021,0.637500,12.734053,0.000000,0.000000,6.375000,39.811862,0.000000,0.000000,0.796875,14.232643,6.375000,39.811862,0.000000,0.000000,4.143750,32.241054,3.346875,29.021570,7.650000,43.499741,0.796875,14.232643,0.000000,0.000000,0.000000,0.000000,12.590625,55.245683,8.446875,45.635550,2.550000,25.372180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1865,xoai,32.681250,55.224753,36.831250,62.237421,26.975000,45.582336,2.55000,25.372180,1.753125,21.070677,9.721875,48.831990,2.709375,26.144787,0.000000,0.000000,23.268750,73.430896,62.475000,109.672236,65.025000,111.144610,35.062500,87.815480,2.231250,23.748479,16.256250,62.298299,24.225000,74.769809,0.478125,11.031467,30.600000,82.865192,23.746875,74.104919,12.750000,55.575962,40.481250,93.187913,3.187500,28.331120,2.550000,25.372180,24.065625,74.549179,10.359375,50.342070,3.506250,29.695117,21.993750,71.586879,31.5

In [ ]:
label_train = train['name']
features_train = train.drop(['name'], axis = 1)
label_valid = valid['name']
features_valid = valid.drop(['name'], axis = 1)

In [ ]:
models = []
models.append(('rfc', RandomForestClassifier()))
models.append(('svc', SVC()))
models.append(('knn', KNeighborsClassifier()))


for name, model in models:
  print(name)
  model.fit(features_train, label_train)
  pred = model.predict(features_valid)
  print('\ttest', accuracy_score(label_valid, pred))

rfc
	test 0.9444444444444444
svc
	test 0.861904761904762
knn
	test 0.7444444444444445


In [ ]:
rfc = RandomForestClassifier()
rfc.fit (features_train, label_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
path = '/content/drive/My Drive/Fruit recognition/valid data '
total_wrong = 0
total_image = 0
file_name = os.listdir(path)
for name in file_name:
  total = 0
  wrong = 0
  print(name)
  for image in os.listdir(path+'/'+name):
    image_path = path+'/'+name+'/'+image
    #print(image_path)
    img = cv2.imread(image_path)
    resized_img = cv2.resize(img, (200, 200))
    cvt_img = cv2.cvtColor(resized_img,cv2.COLOR_BGR2RGB)

    cluster_img, center_color = clustering_image(cvt_img)
    final_cluster = remove_light_color(cluster_img, center_color)

    blur = cv2.GaussianBlur(final_cluster, (3,3), 0)
    edge = cv2.Canny(blur, 70, 50)

    df1 = create_df_with_one_image1(final_cluster)
    df2 = create_df_with_one_image2(edge)
    final_df = pd.concat([df1, df2],axis=1)

    predicted_label = rfc.predict(final_df)[0]
    if predicted_label != name:
      #print('WRONG  -------> {} -------> {}'.format(name, predicted_label))
      wrong+=1
      total_wrong+=1
    #else:
      #print('RIGHT  -------> {}'.format(name))
    total+=1
    total_image+=1

    """figure_size = 6
    plt.figure(figsize=(figure_size,figure_size))

    plt.subplot(1,3,1)
    plt.imshow(cvt_img)
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])

    plt.subplot(1,3,2)
    plt.imshow(final_cluster)
    plt.title('cluster'.format(name, predicted_label)), plt.xticks([]), plt.yticks([])

    plt.subplot(1,3,3)
    plt.imshow(edge, cmap='gray')
    plt.title('Canny edge'.format(name, predicted_label)), plt.xticks([]), plt.yticks([])

    plt.show()"""
  print('{}/{} wrong'.format(wrong, total))

tao
15/16 wrong
nho
8/12 wrong
quyt
3/3 wrong
xoai
19/19 wrong
thom
13/19 wrong
cam
3/19 wrong
chuoi
16/22 wrong
le
20/20 wrong
duahau
16/19 wrong
cachua
2/20 wrong
